In [49]:
# import functions
import idaes
import pandas as pd
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from idaes.core import FlowsheetBlock

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# import HFC_IL mixture properties
from HFC32_emimTf2N_PR import configuration

# Read data
data = pd.read_csv('R32_emimTf2N.csv')

In [50]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": False,
                           "has_pressure_change": True})

DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

2021-03-08 00:13:25 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-08 00:13:25 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-08 00:13:25 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component emimTf2N was not assigned a type. Using generic Component object.
2021-03-08 00:13:25 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-08 00:13:25 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
The initial DOF is 6


In [51]:
#Load data for a single temperature
data_emimTf2N=pd.read_csv('~/Documents/Github/extractive-distillation2/R32_emimTf2N.csv')
print(data_emimTf2N[15:23])

# input infor into model
m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)                # K
m.fs.F101.inlet.pressure.fix(7.003E5)                  # Pa
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.320)
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(1-0.5)
# m.fs.F101.vap_outlet.pressure.fix(399800)

# # DOF analysis
# degrees_of_freedom(m.fs.F101)

         T       P     x1
15  323.15  0.0102  0.005
16  323.15  0.1003  0.055
17  323.15  0.2497  0.133
18  323.15  0.4001  0.200
19  323.15  0.5495  0.264
20  323.15  0.7003  0.320
21  323.15  0.8504  0.368
22  323.15  1.0004  0.417


In [52]:
# Initialize the flash unit
m.fs.F101.initialize()

2021-03-08 00:13:26 [INFO] idaes.init.fs.F101.control_volume.properties_in: Starting initialization
2021-03-08 00:13:26 [INFO] idaes.init.fs.F101.control_volume.properties_in: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-03-08 00:13:26 [INFO] idaes.init.fs.F101.control_volume.properties_in: Equilibrium temperature initialization completed.
2021-03-08 00:13:27 [INFO] idaes.init.fs.F101.control_volume.properties_in: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-03-08 00:13:27 [INFO] idaes.init.fs.F101.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2021-03-08 00:13:27 [INFO] idaes.init.fs.F101.control_volume.properties_out: Starting initialization
2021-03-08 00:13:27 [INFO] idaes.init.fs.F101.control_volume.properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-03-08 00:13:27 [INFO] idaes.init.fs.F101.control_volume.properties_out: Equilibrium temperatur

In [53]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [54]:
status = solver.solve(m, tee = False)

In [55]:
# display solution

m.fs.F101.report()
# m.display()
# m.pprint()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value  : Fixed : Bounds
    Pressure Change : 3293.9 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                               Inlet    Vapor Outlet  Liquid Outlet
    flow_mol                    1.0000      0.12334       0.69666  
    mole_frac_comp emimTf2N    0.50000   1.0000e-08       0.71771  
    mole_frac_comp R32         0.32000       1.0000       0.28229  
    temperature                 323.15       324.91        324.91  
    pressure                7.0030e+05   7.0359e+05    7.0359e+05  
